# OSM Shapefile Decoding

In [2]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-28 09:38:38.058269


In [3]:
# Global Params
basedir="/Users/tgadf/Downloads/osm"
prec=7
mainkey="OSM"

# Parse OSM Data

In [4]:
from ioUtils import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

fastfood=["fastfood", "foodcourt","convenience", "cafe"]
restaurant=["restaurant"]
homefood=["greengrocer", "pharmacy", "supermarket", "chemist"]
lodging=["hotel", "motel", "hostel","chalet","bedandbreakfast","guesthouse", "caravansite"]
commercial=["bookshop","butcher","hairdresser", "furnitureshop", "doityourself", "gardencentre", "newsagent", "toyshop","mall","laundry","mobilephoneshop","bank","atm"]
commercial+=["bicyclerental","bicycleshop","bakery","beautyshop","videoshop","outdoorshop", 'jeweller','kiosk', 'shoeshop','sportsshop', "travelagent"]
commercial+=['florist','giftshop', "departmentstore", "computershop"]
vending=['vendingany','vendingcigarette','vendingmachine','vendingparking']
commercial+=vending
auto=['cardealership','carrental','carrepair','carsharing','carwash']
municipal=["firestation", "prison", "library", "postoffice", "postbox", "police", "townhall","communitycentre","artscentre", "publicbuilding", "courthouse"]
industrial=["wastewaterplant", "windmill","watermill","waterworks", "recycling", "recyclingclothes", "recyclingglass", "recyclingmetal", "recyclingpaper", "commstower", 'watertower','waterwell', "tower"]
building=["embassy"]
medical=["hospital", "dentist", "doctors", "optician", "veterinary","nursinghome"]
attraction=["attraction","fountain", "museum", "touristinfo", "viewpoint", "zoo","themepark","monument","picnicsite","fort","castle","battlefield","archaeological","memorial", "lighthouse", "observationtower", "artwork"]
religious=["graveyard","waysidecross","waysideshrine","ruins"]
sport=["golfcourse", "huntingstand", "icerink", "sportscentre", "track", "swimmingpool"]
entertainment=["bar", "pub", "nightclub", "theatre", "cinema"]
recreation=["stadium", "campsite","biergarten","dogpark","park", "pitch", "playground"]
school=["kindergarten","school"]
college=["college","university"]
general=['alpinehut','bench','beverages','camerasurveillance','clothes','drinkingwater','shelter','stationery','telephone','toilet','wastebasket']                

increase={}

def parseOSM(dirval, prec, key):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*{0}*.shp".format(key)))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    
    
    shapeData   = {}
    geoShapeMap = {}
    ngeos     = 0
    totalgeos = 0
    show      = False
    cntr      = Counter()
        
    print("Found the following {0} shape files: {1}".format(len(osms), ", ".join(osms)))
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, prec)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        Nshapes   = len(sf.shapes())

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 2500 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                if fclass.find("christian_") != -1:
                    fclass = "christian"
                elif fclass.find("muslim_") != -1:
                    fclass = "muslim"
                else:
                    fclass = fclass
                    
                if fclass in ['hindu', 'sikh', 'sihk', 'taoist']:
                    fclass = "other"
                geoid = fclass
            elif osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("bicycle") != -1:
                    fclass = "bicycle"
                elif fclass.find("parking_") != -1:
                    fclass = "parking"
                elif fclass.find("mini_roundabout") != -1 or fclass.find("turning_circle") != -1:
                    fclass = "roundabout"
                elif fclass.find("speed_camera") != -1:
                    fclass = "speedcamera"
                elif fclass.find("motorway_junction") != -1:
                    fclass = "ramp"
                elif fclass.find("lock_gate") != -1:
                    fclass = "gate"
                elif fclass.find("traffic_signals") != -1:
                    fclass = "signal"
                elif fclass.find("street_lamp") != -1:
                    fclass = "lamp"
                else:
                    fclass = fclass
                    
                if fclass not in ["parking","fuel"]:
                    continue
                geoid = fclass
            elif osm in ["gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("railway_") != -1:
                    fclass = fclass.replace("railway", "rail")
                fclass = fclass.replace("_", "")
                
                if fclass.startswith("bus"):
                    fclass = "bus"
                elif fclass.startswith("rail"):
                    fclass = "rail"
                elif fclass.startswith("ferry"):
                    fclass = "ferry"
                elif fclass.startswith("tram"):
                    fclass = "tram"
                elif fclass.startswith("taxi"):
                    fclass = "taxi"
                else:
                    raise ValueError("fClass {0} was not recognized!".format(fclass))
                
                geoid = fclass
            elif osm in ["gis_osm_pois_a_free_1", "gis_osm_pois_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                fclass = fclass.replace("_", "")
                
                if fclass in fastfood:
                    fclass = "fastfood"
                elif fclass in restaurant:
                    fclass = "restaurant"
                elif fclass in homefood:
                    fclass = "grocery"
                elif fclass in lodging:
                    fclass = "lodging"
                elif fclass in commercial:
                    fclass = "business"
                elif fclass in auto:
                    fclass = "auto"
                elif fclass in municipal:
                    fclass = "municipal"
                elif fclass in industrial:
                    fclass = "manufacturing"
                elif fclass in building:
                    fclass = "building"
                elif fclass in medical:
                    fclass = "medical"
                elif fclass in attraction:
                    fclass = "attraction"
                elif fclass in religious:
                    fclass = "religious"
                elif fclass in sport:
                    fclass = "sport"
                elif fclass in entertainment:
                    fclass = "entertainment"
                elif fclass in recreation:
                    fclass = "recreation"
                elif fclass in school:
                    fclass = "school"
                elif fclass in college:
                    fclass = "college"
                elif fclass in general:
                    fclass = "public"
                else:
                    raise ValueError("fClass {0} was not found!".format(fclass))
                geoid = fclass
            elif osm in ["gis_osm_water_a_free_1", "gis_osm_waterways_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_natural_a_free_1", "gis_osm_natural_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_landuse_a_free_1", "gis_osm_landuse_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]                
                geoid = fclass
            elif osm in ["gis_osm_railways_a_free_1", "gis_osm_railways_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]                
                geoid = fclass
            elif osm in ["gis_osm_roads_a_free_1", "gis_osm_roads_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                
                if fclass.find("primary") != -1:
                    fclass = "primary"
                if fclass.find("secondary") != -1:
                    fclass = "seconday"
                if fclass.find("tertiary") != -1:
                    fclass = "tertiary"
                if fclass.find("trunk") != -1:
                    fclass = "trunk"
                if fclass.find("track") != -1:
                    fclass = "track"
                if fclass.find("motorway") != -1:
                    fclass = "motorway"
                if fclass.find("living_street") != -1:
                    fclass = "residential"
                if fclass in ["unclassified", "unknown", "bridleway"]:
                    fclass = "other"
                
                geoid = fclass
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #print(irec,fclass,geoid)
            #continue
            #if irec > 1000: 1/0

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            
            geos = getGeos(shape, prec=prec, linear=False, returnKeys=True, debug=False)

            if False:
                useLinear = False
                if len(points) == 1 or useLinear is True:
                    geos = getGeos(shape, prec=prec, linear=True, returnKeys=True)
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
                else:
                    geos = getInitGeo(shape)
                    geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                    if geos is None:
                        useLinear = True
                        geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = 1
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        print("Finished: {0}".format(cntr.most_common(100)))
        print("\n")
            
    found = False
    for k,v in cntr.most_common():
        if k.find(" ") != -1 or k.find(";") != -1 or k[0].isupper():
            print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
            found = True
    print("\n")
    for k,v in nameCntr.most_common(200):
        print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
    if len(shapeData) > 0:
        ncategories = len(geoShapeMap)
        if ncategories == 1:
            for geoid in geoShapeMap.keys():
                geoShapeMap[geoid] = list(set(geoShapeMap[geoid].keys()))
            print("Found {0} geos from {1}".format(ngeos, shapeval))
            saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, key, prec)))
        else:
            for geoid in geoShapeMap.keys():
                shapeDataSave = {}
                shapeDataSave[geoid] = shapeData[geoid]
                geoShapeMapSave = {}
                geoShapeMapSave[geoid] = list(set(geoShapeMap[geoid].keys()))
                ngeos = len(geoShapeMapSave[geoid])
                print("Found {0} geos from {1} for {2}".format(ngeos, shapeval, geoid))
                saveGeoData(shapeDataSave, geoShapeMapSave, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}-{3}".format(state, key, geoid, prec)))

In [5]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    
    try:
        rmZipDir(dirval)
    except:
        pass
    
    
    if exists(dirval):
        print("Directory {0} already exists".format(dirval))
        rmZipDir(dirval)
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    print("Extract {0}".format(zipname))
    zip_ref.extractall(dirval)
    zip_ref.close()
    
    #try:
    parseOSM(dirval, 7, 'natural')
    rmZipDir(dirval)

Extract /Users/tgadf/Downloads/osm/alabama-latest-free.shp.zip
Found the following 2 shape files: gis_osm_natural_a_free_1, gis_osm_natural_free_1
gis_osm_natural_a_free_1
Current Time is Wed Nov 28, 2018 09:38:44 for 

Analyzing gis_osm_natural_a_free_1	41
Finished: [('beach', 41)]


gis_osm_natural_free_1
Current Time is Wed Nov 28, 2018 09:38:44 for 

Analyzing gis_osm_natural_free_1	3763
Processed 2500/3763 records. Found 2665 geos so far...
Finished: [('tree', 2732), ('peak', 851), ('cliff', 167), ('beach', 44), ('spring', 9), ('cave_entrance', 1)]




Found 162 geos from gis_osm_natural_free_1 for beach


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/alabama-natural-beach-7-data.p


There are 162 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/alabama-natural-beach-7-geos.p


Found 850 geos from gis_osm_natural_free_1 for peak


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/a

Saved shape data to /Users/tgadf/Downloads/osm/arizona-natural-cave_entrance-7-data.p


There are 34 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/arizona-natural-cave_entrance-7-geos.p


Found 83 geos from gis_osm_natural_free_1 for volcano


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/arizona-natural-volcano-7-data.p


There are 83 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/arizona-natural-volcano-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/arizona-latest-free.shp
Extract /Users/tgadf/Downloads/osm/arkansas-latest-free.shp.zip
Found the following 2 shape files: gis_osm_natural_a_free_1, gis_osm_natural_free_1
gis_osm_natural_a_free_1
Already processed this one...
Current Time is Wed Nov 28, 2018 09:39:14 for 

Analyzing gis_osm_natural_a_free_1	51
Finished: [('beach', 50), ('cliff', 1)]


gis_osm_natural_free_1
Already processed this one...
Current Time is Wed Nov 28, 2018

Saved shape data to /Users/tgadf/Downloads/osm/california-natural-spring-7-data.p


There are 1257 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-spring-7-geos.p


Found 6844 geos from gis_osm_natural_free_1 for peak


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-peak-7-data.p


There are 6844 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-peak-7-geos.p


Found 13270 geos from gis_osm_natural_free_1 for tree


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-tree-7-data.p


There are 13270 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-tree-7-geos.p


Found 122 geos from gis_osm_natural_free_1 for cave_entrance


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/california-natural-cave_entrance-7-data

Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-beach-7-data.p


There are 180 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-beach-7-geos.p


Found 7 geos from gis_osm_natural_free_1 for cave_entrance


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-cave_entrance-7-data.p


There are 7 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-cave_entrance-7-geos.p


Found 8765 geos from gis_osm_natural_free_1 for tree


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-tree-7-data.p


There are 8765 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-tree-7-geos.p


Found 3269 geos from gis_osm_natural_free_1 for peak


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/colorado-natural-peak-7-data.p


There are 

Processed 25000/40416 records. Found 34988 geos so far...
Processed 27500/40416 records. Found 37488 geos so far...
Processed 30000/40416 records. Found 39988 geos so far...
Processed 32500/40416 records. Found 42488 geos so far...
Processed 35000/40416 records. Found 44988 geos so far...
Processed 37500/40416 records. Found 47488 geos so far...
Processed 40000/40416 records. Found 49988 geos so far...
Finished: [('tree', 39322), ('spring', 756), ('beach', 649), ('peak', 155), ('cliff', 54), ('cave_entrance', 19), ('volcano', 1)]




Found 9723 geos from gis_osm_natural_free_1 for beach


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/florida-natural-beach-7-data.p


There are 9723 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/florida-natural-beach-7-geos.p


Found 634 geos from gis_osm_natural_free_1 for spring


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/florida-natural-sprin

...

Processed 12500/14737 records. Found 18011 geos so far...
Finished: [('tree', 9967), ('peak', 4014), ('spring', 475), ('beach', 216), ('cliff', 177), ('volcano', 64), ('glacier', 50), ('cave_entrance', 17)]




Found 4170 geos from gis_osm_natural_free_1 for beach


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-natural-beach-7-data.p


There are 4170 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-natural-beach-7-geos.p


Found 1179 geos from gis_osm_natural_free_1 for glacier


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-natural-glacier-7-data.p


There are 1179 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-natural-glacier-7-geos.p


Found 904 geos from gis_osm_natural_free_1 for tree


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-natural-tree-7-data.p


There are 904 entries 

Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-beach-7-data.p


There are 932 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-beach-7-geos.p


Found 10 geos from gis_osm_natural_free_1 for cliff


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-cliff-7-data.p


There are 10 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-cliff-7-geos.p


Found 235 geos from gis_osm_natural_free_1 for peak


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-peak-7-data.p


There are 235 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-peak-7-geos.p


Found 206 geos from gis_osm_natural_free_1 for tree


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/rhode-island-natural-tree-7-data.p


There a

Found the following 2 shape files: gis_osm_natural_a_free_1, gis_osm_natural_free_1
gis_osm_natural_a_free_1
Current Time is Wed Nov 28, 2018 09:43:33 for 

Analyzing gis_osm_natural_a_free_1	135
Finished: [('beach', 118), ('peak', 6), ('cave_entrance', 3), ('cliff', 3), ('spring', 3), ('tree', 2)]


gis_osm_natural_free_1
Current Time is Wed Nov 28, 2018 09:43:34 for 

Analyzing gis_osm_natural_free_1	37281
Processed 2500/37281 records. Found 5062 geos so far...
Processed 5000/37281 records. Found 7562 geos so far...
Processed 7500/37281 records. Found 10062 geos so far...
Processed 10000/37281 records. Found 12562 geos so far...
Processed 12500/37281 records. Found 15062 geos so far...
Processed 15000/37281 records. Found 17562 geos so far...
Processed 17500/37281 records. Found 20062 geos so far...
Processed 20000/37281 records. Found 22562 geos so far...
Processed 22500/37281 records. Found 25062 geos so far...
Processed 25000/37281 records. Found 27562 geos so far...
Processed 275

Found the following 2 shape files: gis_osm_natural_a_free_1, gis_osm_natural_free_1
gis_osm_natural_a_free_1
Current Time is Wed Nov 28, 2018 09:43:49 for 

Analyzing gis_osm_natural_a_free_1	93
Finished: [('beach', 93)]


gis_osm_natural_free_1
Current Time is Wed Nov 28, 2018 09:43:50 for 

Analyzing gis_osm_natural_free_1	18706
Processed 2500/18706 records. Found 4939 geos so far...
Processed 5000/18706 records. Found 7439 geos so far...
Processed 7500/18706 records. Found 9939 geos so far...
Processed 10000/18706 records. Found 12439 geos so far...
Processed 12500/18706 records. Found 14939 geos so far...
Processed 15000/18706 records. Found 17439 geos so far...
Processed 17500/18706 records. Found 19939 geos so far...
Finished: [('tree', 16596), ('peak', 1961), ('beach', 132), ('cliff', 68), ('spring', 37), ('cave_entrance', 5)]




Found 2452 geos from gis_osm_natural_free_1 for beach


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/virginia

Saved shape data to /Users/tgadf/Downloads/osm/west-virginia-natural-spring-7-data.p


There are 3 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/west-virginia-natural-spring-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/west-virginia-latest-free.shp
Extract /Users/tgadf/Downloads/osm/wisconsin-latest-free.shp.zip
Found the following 2 shape files: gis_osm_natural_a_free_1, gis_osm_natural_free_1
gis_osm_natural_a_free_1
Current Time is Wed Nov 28, 2018 09:44:04 for 

Analyzing gis_osm_natural_a_free_1	418
Finished: [('beach', 413), ('spring', 3), ('cliff', 2)]


gis_osm_natural_free_1
Current Time is Wed Nov 28, 2018 09:44:05 for 

Analyzing gis_osm_natural_free_1	23903
Processed 2500/23903 records. Found 4324 geos so far...
Processed 5000/23903 records. Found 6824 geos so far...
Processed 7500/23903 records. Found 9324 geos so far...
Processed 10000/23903 records. Found 11824 geos so far...
Processed 12500/23903 records. Found 14324 geos so far...

In [24]:
# gis_osm_buildings_a_free_1.shp	gis_osm_pofw_a_free_1.shp	gis_osm_traffic_a_free_1.shp
# gis_osm_landuse_a_free_1.shp	gis_osm_pofw_free_1.shp		gis_osm_traffic_free_1.shp
# gis_osm_natural_a_free_1.shp	gis_osm_pois_a_free_1.shp	gis_osm_transport_a_free_1.shp
# gis_osm_natural_free_1.shp	gis_osm_pois_free_1.shp		gis_osm_transport_free_1.shp
# gis_osm_places_a_free_1.shp	gis_osm_railways_free_1.shp	gis_osm_water_a_free_1.shp
# gis_osm_places_free_1.shp	gis_osm_roads_free_1.shp	gis_osm_waterways_free_1.shp